In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
import csv

from selenium import webdriver
from selenium.common.exceptions import TimeoutException
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By

In [ ]:
path = "./chromedriver_win32/chromedriver.exe"
driver = webdriver.Chrome(path)

In [ ]:
def CrawlByPeriod(keyword, startDate, endDate):
    # 수집된 기사의 링크 목록
    newsSiteList = []
    url = "https://search.naver.com/search.naver?&where=news"
    url = url + "&query=" + query
    url = url + "&sm=tab_pge&sort=0&photo=0&field=0&reporter_article=&pd=3"
    url = url + "&ds=" + startDate + "&de=" + endDate
    url = url + "&docid=&nso=so:r,p:from" + startDate.replace(".","") + "to"+ endDate.replace(".","") + ",a:all&mynews=0&cluster_rank=29&start=1&refresh_start=0"
    
    driver.get(url)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    #print(soup.prettify())
    newsSiteList = soup.find_all('a', '_sp_each_url')
    
    newsCommentList = []
    for newsIndex in range(len(newsSiteList)):
        driver.get(newsSiteList[newsIndex]['href']+"&m_view=1") # m_view=1 을 붙이면 댓글 창으로 이동
        timeout = 5
        try:
            element_present = EC.presence_of_element_located((By.ID, 'main'))
            WebDriverWait(driver, timeout).until(element_present)
        except TimeoutException:
            print("Timed out waiting for page to load")
        finally:
            print("Page loaded")
        
        html = driver.page_source
        soup = BeautifulSoup(html, 'html.parser')
        #print(soup.prettify())
        newsCommentList = newsCommentList + soup.find_all('span', 'u_cbox_contents')
         
    return newsCommentList

In [ ]:
def CrawlByArticleUrl(url):

    driver.get(url+"&m_view=1") # m_view=1 을 붙이면 댓글 창으로 이동
    timeout = 5
    try:
        element_present = EC.presence_of_element_located((By.ID, 'main'))
        WebDriverWait(driver, timeout).until(element_present)
    except TimeoutException:
        print("Timed out waiting for page to load")
    finally:
        print("Page loaded")
    
    try:
        while 1:
            #button = driver.find_element_by_css_selector(".u_cbox_page_more")
            #driver.implicitly_wait(10)
            button = WebDriverWait(driver, 10).until(EC.element_to_be_clickable((By.CSS_SELECTOR, '.u_cbox_page_more')))
            button.click()
            
    except Exception as e:
        print(e)
        
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    #print(soup.prettify())
    newsCommentList = soup.find_all('span', 'u_cbox_contents')
         
    return newsCommentList

In [ ]:
# 여기에 검색 키워드와 기간을 입력
keyword = "여행"
startDate = "2018.02.01"
endDate = "2018.02.01"
newsCommentList = CrawlByPeriod(query, start, end)
print(len(newsCommentList))

In [ ]:
newsCommentList = CrawlByArticleUrl("https://news.naver.com/main/ranking/read.nhn?mid=etc&sid1=111&rankingType=popular_day&oid=023&aid=0003561349&date=20200912&type=1&rankingSeq=9&rankingSectionId=100")
newsCommentList += CrawlByArticleUrl("https://news.naver.com/main/ranking/read.nhn?rankingType=popular_memo&oid=025&aid=0003034634&date=20200913&type=1&rankingSectionId=102&rankingSeq=1")
print(len(newsCommentList))

In [ ]:
# 댓글을 csv 파일로 저장
csvFilename = 'NewsComment.csv'
csvfile = open(csvFilename, 'a', newline='', encoding='utf-8-sig')
csvwriter = csv.writer(csvfile, delimiter='\t')

for cmtIndex in range(len(newsCommentList)):
    print(cmtIndex, newsCommentList[cmtIndex].text)
    csvwriter.writerow(newsCommentList[cmtIndex].text)
    
csvfile.close()